In [10]:
import cv2
from yolo_segmentation import YOLOSegmentation
import numpy as np

In [11]:
def rszImg(image, rate):
    h, w, _ = image.shape
    new_h, new_w = (int(h*rate), int(w*rate))
    image = cv2.resize(image, (new_w, new_h))
    return image

def cropPoly(img, bbox, seg):
    (x, y, x2, y2) = bbox
    # Create a mask of the same size as the image
    mask = np.zeros_like(img)

    # Convert the seg (polygon) into a NumPy array of type int32
    points = np.array(seg, dtype=np.int32)

    # Fill the mask with a white polygon based on the points
    cv2.fillPoly(mask, [points], (255, 255, 255))

    # Bitwise AND the mask with the original image to get the cropped region
    cropped_motorcycle = cv2.bitwise_and(img, mask)

    # Crop the region inside the bounding box
    cropped_motorcycle = cropped_motorcycle[y:y2, x:x2]
    
    return cropped_motorcycle

In [12]:

path_nyala = '../dataset-baru/beat-nyala/1.jpg'
# path_nyala = 'awikwok.jpg'
path_mati = '../dataset-baru/beat-mati/0.jpg'


# flag = 'nyala'
# img = cv2.imread(path_nyala)
flag = 'mati'
img = cv2.imread(path_mati)

# img = cv2.resize(img, None, fx=0.2, fy=0.2)
img = rszImg(img, 0.25)

In [13]:
# Segmentation
# ys = YOLOSegmentation('../../yolov8n-seg.pt')
ys = YOLOSegmentation('../learn/yolov8-polylines/yolov8n-seg.pt') 
bboxes, classes, class_names, segmentation, scores = ys.detect(img)

print(class_names)


0: 640x480 1 motorcycle, 1 skateboard, 473.3ms
Speed: 13.6ms preprocess, 473.3ms inference, 38.7ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.


{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [14]:
cropped_motorcycle_regions = []
n = 0

for bbox, class_id, seg, score in zip(bboxes, classes, segmentation, scores):
    # print("bbox:", bbox, " class_id:", class_id, " seg:", seg, " score:", score)
    if class_names[class_id] == 'motorcycle':
        (x, y, x2, y2) = bbox

        cropped_motorcycle = cropPoly(img, bbox, seg)
        cropped_motorcycle_regions.append(cropped_motorcycle)

        # Draw the bounding box, polyline, and class name on the original image
        cv2.rectangle(img, (x, y), (x2, y2), (0, 0, 255), 2)
        cv2.polylines(img, [seg], True, (255, 0, 0), 2)
        cv2.putText(img, str(class_names[class_id]), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)

In [15]:
# Save the cropped motorcycle regions or do further processing
for i, cropped_motorcycle in enumerate(cropped_motorcycle_regions):
    if flag == 'nyala':
        cv2.imwrite(f'result/nyala/{i}.jpg', cropped_motorcycle)
    else :
        cv2.imwrite(f'result/mati/{i}.jpg', cropped_motorcycle)
        

In [16]:
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()